In [2]:
import rethinkdb as r

In [3]:
import pandas as pd
import numpy as np

In [4]:
conn = r.connect(
 host="localhost",
 port=28015,
 db='binance'
)

['LINK', 'ADA', 'EOS', 'STRAT', 'TNB']

In [89]:
1553775420000-1553775240000

180000

In [118]:
cursor = r.table('features').filter(lambda doc:
            r.expr(['XRP'])
                .contains(doc['baseAsset'])
        ).limit(10).order_by(r.desc('startTime')).pluck('startTime','asks', 'bids').run(conn)

In [119]:
books = []
for book in cursor:
    books.append(book)

In [120]:
books

[{'asks': [{'price': 7.588e-05, 'quantity': 13634},
   {'price': 7.589e-05, 'quantity': 8000},
   {'price': 7.59e-05, 'quantity': 1136},
   {'price': 7.592e-05, 'quantity': 5223},
   {'price': 7.593e-05, 'quantity': 4549},
   {'price': 7.594e-05, 'quantity': 26614},
   {'price': 7.595e-05, 'quantity': 33946},
   {'price': 7.596e-05, 'quantity': 17260},
   {'price': 7.597e-05, 'quantity': 19404},
   {'price': 7.598e-05, 'quantity': 11859},
   {'price': 7.599e-05, 'quantity': 7594},
   {'price': 7.6e-05, 'quantity': 675289},
   {'price': 7.601e-05, 'quantity': 5984},
   {'price': 7.602e-05, 'quantity': 240},
   {'price': 7.603e-05, 'quantity': 278},
   {'price': 7.604e-05, 'quantity': 1768},
   {'price': 7.605e-05, 'quantity': 2410},
   {'price': 7.606e-05, 'quantity': 792},
   {'price': 7.607e-05, 'quantity': 839},
   {'price': 7.608e-05, 'quantity': 24919}],
  'bids': [{'price': 7.587e-05, 'quantity': 14},
   {'price': 7.585e-05, 'quantity': 418},
   {'price': 7.584e-05, 'quantity': 99

In [125]:
def derive_diff(depth, prev_depth, side, levels=5):
    diff = []
    assert levels <= len(books[0]['asks'])
    for x in range(levels):
        diff.append((depth[side][x]['quantity'] - prev_depth[side][x]['quantity']) *  depth[side][x]['price'])
    return diff

In [128]:
def derive_piff(depth, prev_depth, side, levels=5):
    diff = []
    assert levels <= len(books[0]['asks'])
    for x in range(levels):
        diff.append(depth[side][x]['price'] - prev_depth[side][x]['price'])
    return diff

In [126]:
derive_diff(books[1], books[0], 'bids', levels=20)

[0.0,
 0.0,
 0.0,
 0.0,
 0.7324212,
 0.0,
 0.249761,
 0.0,
 -0.71930376,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [134]:
x=derive_piff(books[1], books[0], 'asks', levels=2)

In [135]:
x[0]-x[1]

-1.3552527156068805e-20

In [90]:
assets = ['BNB', 'ETH', 'XRP']

In [95]:
def get_last_time():
        # Get the last end time 
        last_event_time = r.table('features')\
            .max('epochEndTime')\
            .pluck('endTime')\
            .run(conn)

        last_time = (last_event_time['endTime']+1)/1000

        return last_time

def get_first_time():
    # Get the last end time 
    first_event_time = r.table('features')\
        .min('epochStartTime')\
        .pluck('startTime')\
        .run(conn)

    first_time = (first_event_time['startTime']+1)/1000

    return first_time

In [136]:
last_time = get_last_time()

# Derive the start time as a function of the 
# end time minus the window size
start = round(last_time - (int(90)*60))
index_columns = [
    'baseAsset',
    'startTime'
]

In [137]:
time_index = pd.to_datetime(list(range(start, round(last_time), 60)),unit='s')
time_index = time_index.round('min')

In [106]:
rec = r.table('features')\
.between(
    r.epoch_time(start),
    r.epoch_time(last_time),
    index='epochEventTime'
)\
.pluck(
    'baseAsset',
    'startTime',
    'close',
    'open',
    'high',
    'low',
    'bids',
    'asks',
    'trades'
)\
.filter(lambda doc:
    r.expr(assets)
        .contains(doc["baseAsset"])
)\
.run(conn)

f = pd.DataFrame(rec)
f['startTime'] = pd.to_datetime(f['startTime'], unit='ms')
f.set_index(index_columns, inplace=True)

# Reindex the dataframe based upon 
ind = pd.MultiIndex.from_product(
    [
        f.index.levels[0],
        time_index
    ],
    names=index_columns
)    
f = f.reindex(ind)

# Fill non existent values
f=f.fillna(axis=0, method="ffill")\
   .fillna(axis=0, method="bfill")

In [109]:
f

asks  \
baseAsset startTime                                                                
BNB       2019-03-28 13:17:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:18:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:19:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:20:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:21:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:22:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:23:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:24:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:25:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:26:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:27:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:28:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:29:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:30:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:31:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:32:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:33:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:34:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:35:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:36:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:37:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:38:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:39:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:40:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:41:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:42:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:43:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:44:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:45:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
          2019-03-28 13:46:00  [{'price': 0.0041421, 'quantity': 54.36}, {'pr...   
...                                                                          ...   
XRP       2019-03-28 14:17:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:18:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:19:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:20:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:21:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:22:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:23:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:24:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:25:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:26:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:27:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:28:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:29:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:30:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:31:00  [{'price': 7.588e-05, 'quantity': 658}, {'pric...   
          2019-03-28 14:32:00  [{'price': 7.

-21592